# K-Server Problem

The K-Server problem consists of a set of servers with some initial position and a sequence of requests to which one of the servers need to be send to before seeing the next request. The total cost, which is to be minimized, is the cumulative distance travelled by the servers.

## This implementation

The following implementation provides several offline and online methods to solve K-Server problem instances. Amongst which are:

### Optimal offline Solvers
- one which uses dynammic programming (rather slow for big instances)
- one which transforms the problem into a max-flow-min-cost problem and solves it using the min-mean-cycle-reduction algorithm

### Online Solvers:
- one which uses random assignment
- one which uses a greedy strategy by always assigning the closest server to the request
- one which uses the Work-Function-Algorithm; note that the problem to solve the cost function of the algorithm also gets solved by transforming it into a max-flow-min-cost problem (see [Rudec et al. 2011](https://link.springer.com/article/10.1007/s10100-011-0222-7))
- one which uses the Follow-the-Prediction algorithm using predicted configurations (see [Polak et al. 2020](https://arxiv.org/abs/2003.02144))
- one which deterministically combines multiple algorithms (Called Min_det in (see [Polak et al. 2020](https://arxiv.org/abs/2003.02144)))
- one which in a randomized way combines two algorithms (Called Min_Rand in (see [Polak et al. 2020](https://arxiv.org/abs/2003.02144)))

### Additional methods
It also provides a method for computing predictions for the Follow-the-Prediction algorithm using one of the other solvers with some error probability. They can however also be stated manually.

There are of course also several other helper methods in addition to the implementation of networks, the Edward-Karps algorithm, Kosarajus algorithm and the min-mean-cycle-reduction algorithm itself. They are however only indirectly part of the K-Server problem and are thus not presented in detail. 

## Example 1

Lets first import the neccessary functions and classes:

In [14]:
import K_Server as ks

To instantiate a K-Server instance, we need some location object for the server and request positions, which implements a distance function. The following example uses points on the line.\
You can however choose arbitrary locations, as long as there is always a distance function implemented like this: 'def dist(self, other) -> float'. This also allows not just for arbitrary dimensions but of course also for any distance metric.

In [15]:
class Point_on_Line:
    """
    A small class simulating a location of a server or request on the line

    ...

    Attributes
    ----------
    x : int
        the position on the line

    Methods
    -------
    dist(other)
        computes the distance between this location and another one 
    
    """
    
    def __init__(self, x: float) -> None:
        self.x = x
        
    def __str__(self) -> str:
        return f'{self.x}'
    
    def __repr__(self) -> str:
        return f'{self.x}'
        
    def dist(self, other) -> float:
        return abs(self.x - other.x)

An instance of the K-Server problem gets instantiated by providing a list of initial server positions, a list of request positions and, if desired, also a list with the predicted configurations. You can also state the metric space diameter, but if you should decide not to, it will get computed automatically.    

In [16]:
# four servers located at 0, -15, 0 & 7
servers = [Point_on_Line(0), Point_on_Line(-15), Point_on_Line(0), Point_on_Line(7)]


# eight requests at 11, 9, 18, 7, 25, -7, -8 & -3
requests = [Point_on_Line(11), Point_on_Line(9), Point_on_Line(18), Point_on_Line(7), Point_on_Line(25), 
            Point_on_Line(-7), Point_on_Line(-8), Point_on_Line(-3)]

Lets have a look at configurations for the case that you decide to provide the list of predicted configurations manually:\
A configuration consists of a list which contains for each server an index of its position. Note that only initial server positions or request positions are possible locations for a server. Thus the index refers to a list of possible positions \[$s_1, ..., s_n, r_1, ..., r_m$\]  of the server positions followed by the request positions.

For example:

In [17]:
prediction_list = [[0, 1, 2, 4], [0, 5, 2, 4], [6, 5, 2, 4], [7, 5, 2, 4], [7, 5, 2, 8], [7, 9, 2, 8], 
                   [7, 10, 2, 8], [7, 10, 11, 8]]

Now we can define our problem instance:

In [18]:
problem = ks.K_Server(servers, requests, prediction_list)

### Offline and online solutions

As mentioned above there are several offline and online solvers implemented. A list of all of them can be found below. Apart from the two which combine multiple algorithms, they do not need any arguments.

- execute_opt_dp() : optimal offline solution of the problem instance via dynamic programming
- execute_opt_network() : optimal offline solution of the problem instance by solving a max-flow-min-cost problem instance
- execute_random() : online solution of the problem instance using random server assignment
- execute_greedy() : online solution of the problem instance using a greedy heuristic
- execute_ftp() : online solution of the problem instance via the FollowThePrediction-algorithm
- execute_wfa() : online solution of the problem instance via work function algorithm
- execute_combine_deterministic(solver) : online solution of the problem instance by deterministically combining different solvers
- execute_combine_randomized(solver, eps) : online solution of the problem instance by combining two solvers in a randomized way

Lets try them out:

In [19]:
print("opt_dp: ", problem.execute_opt_dp())
print("opt_net: ", problem.execute_opt_network())
print("random: ", problem.execute_random())
print("greedy: ", problem.execute_greedy())
print("ftp: ", problem.execute_ftp())
print("wfa: ", problem.execute_wfa())

opt_dp:  (41, [0, 0, 0, 3, 0, 1, 1, 2])
opt_net:  (41, [3, 0, 3, 0, 3, 1, 1, 2])
random:  (111, [1, 3, 0, 0, 3, 0, 1, 1])
greedy:  (42, [3, 3, 3, 0, 3, 2, 2, 2])
ftp:  (91, [3, 1, 0, 0, 3, 1, 1, 2])
wfa:  (56, [3, 3, 3, 0, 3, 2, 0, 0])


The resulting tuple first contains the total cost (you can see that the total cost of the offline algorithms is the same and also the lowest one), and second a server assignment list, which contains for each request the index of the respective server which serves it.\
This also allows to compare how the algorithms got to their result. For example the two offline algorithms choose for several requests different servers, but both of those solutions are still optimal.

### Combining algorithms

There are two more online algorithms we haven't tried out yet. Both of them are able to either combine two or arbitrarily many other approaches.\
You can state which approaches to use by passing a list of the respesctive solvers as an argument. A list of all availlable solvers can be found below. Note that a solver (in this implementation) refers to the method, which assigns for a given current configuration and the current request a server to serve that request. The "execute_" methods use those solvers.

- opt_solver : optimal solver returning an index of the server to be moved to the current request 
- random_solver : solver for random assignment returning an index of the server to be moved to the current request
- greedy_solver : solver for the greedy algorithm returning an index of the server to be moved to the current request
- ftp_solver : solver for FollowThePrediction returning an index of the server to be moved to the current request
- wfa_network_solver : solver for the work function algorithm returning an index of the server to be moved to the current request

The deterministic combination algorithm can combine arbitrarily many, while the randomized one can (in this implementation) only combine two.\
Also note that the randomized algorithm requires an $\epsilon < 0.5$ as a parameter too (as stated in [Polak et al. 2020](https://arxiv.org/abs/2003.02144)))

In [20]:
# Example for the deterministic algorithm:

# list of solvers to use for the deterministic algorithm
solver_for_det = [problem.ftp_solver, problem.wfa_network_solver, problem.opt_solver]

print("deterministic combination of ftp, wfa and the optimal solver: ", 
      problem.execute_combine_deterministic(solver_for_det))

deterministic combination of ftp, wfa and the optimal solver:  56


In [21]:
# Example for the randomized algorithm:

# list of solvers to use for the randomized algorithm
solver_for_ran = [problem.ftp_solver, problem.greedy_solver]
eps = 0.01

print("randomized combination of ftp and the greedy solver: ", problem.execute_combine_randomized(solver_for_ran, eps))

randomized combination of ftp and the greedy solver:  91


Note that in those two cases only the output only consists of the total cost, since by switching between algorithms servers sometimes also might be moved when not directly serving a request.   

If you do not want to state the predictions manually but instead use one of the solvers to generate them, you can use the following method:

generate_prediction_list(solver, error_prob)

Similar to when combining algorithms you pass a solver as an argument. The predictions will automatically be stored as an attribute of your problem instance.\
The 'error_prob' argument is the probability with which the predicted configuration will deviate from the one suggested by the solver. The default is 0.

In [22]:
# Example
problem.generate_prediction_list(solver = problem.opt_solver, error_prob=0.3)

print("ftp: ", problem.execute_ftp())

ftp:  (79, [0, 0, 3, 0, 0, 2, 1, 3])


## Example 2

A second example where I do not explain everything in detail:

In [23]:
import math

class Point_2D:
    """
    A small class simulating a location of a server or request on the plane

    ...

    Attributes
    ----------
    x : int
        x position
    x : int
        y position

    Methods
    -------
    dist(other)
        computes the distance between this location and another one 
    
    """
    
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y
        
    def __str__(self) -> str:
        return f'{self.x}'
    
    def __repr__(self) -> str:
        return f'{self.x}'
        
    def dist(self, other) -> float:
        return abs(math.sqrt((self.x-other.x)**2 + (self.y-other.y)**2))

In [24]:
servers = [Point_2D(0,0), Point_2D(10,1)]


requests = [Point_2D(4,5), Point_2D(7,2), Point_2D(-2,0), Point_2D(5,7), Point_2D(1,2), Point_2D(10,9), 
            Point_2D(0,5), Point_2D(15,-1)]

problem = ks.K_Server(servers, requests)

problem.generate_prediction_list(solver = problem.greedy_solver)

print("opt_dp: ", problem.execute_opt_dp())
print("opt_net: ", problem.execute_opt_network())
print("random: ", problem.execute_random())
print("greedy: ", problem.execute_greedy())
print("ftp: ", problem.execute_ftp())
print("wfa: ", problem.execute_wfa())

opt_dp:  (42.17224167544759, [1, 1, 0, 1, 0, 1, 0, 1])
opt_net:  (42.17224167544759, [1, 1, 0, 1, 0, 1, 0, 1])
random:  (71.25466026528898, [0, 1, 1, 1, 0, 0, 0, 0])
greedy:  (46.09415001090821, [0, 1, 0, 1, 0, 1, 0, 1])
ftp:  (46.09415001090821, [0, 1, 0, 1, 0, 1, 0, 1])
wfa:  (46.09415001090821, [0, 1, 0, 1, 0, 1, 0, 1])


In [25]:
solver_for_det = [problem.ftp_solver, problem.wfa_network_solver, problem.opt_solver]

print("deterministic combination of ftp, wfa and the optimal solver: ", 
      problem.execute_combine_deterministic(solver_for_det))

solver_for_ran = [problem.ftp_solver, problem.greedy_solver]
eps = 0.01

print("randomized combination of ftp and the greedy solver: ", problem.execute_combine_randomized(solver_for_ran, eps))

deterministic combination of ftp, wfa and the optimal solver:  54.38561558878709
randomized combination of ftp and the greedy solver:  46.09415001090821


Just as a small remark: while testing the second example it at first did not work due to a floating point error. I changed one line in the code so it does work now, but theoretically something similar could happen again, as I did not directly watch out for something like that when writing the code.